# Demonstrate Interaction of pydantic and omnikeeper

In [1]:
import os
import omnikeeper_client as okc
from omnikeeper_client import TraitDefinition, TraitAttributeDefinition, ATTRIBUTETYPE_TEXT, ATTRIBUTETYPE_INTEGER
import uuid
from pydantic import BaseModel,TypeAdapter
from typing_extensions import Annotated
from typing import List

In [2]:
trait_id = "python_client_demo.test"
layer_id = "testlayer"

create a omnikeeper client

In [3]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

ensure trait exists

In [4]:
ret = okc.upsert_trait(okapiclient, TraitDefinition(trait_id, [
        TraitAttributeDefinition("id", "test.id", ATTRIBUTETYPE_INTEGER),
        TraitAttributeDefinition("array", "test.array", ATTRIBUTETYPE_TEXT, is_array=True),
        TraitAttributeDefinition("some_key", "test.mapped_some_key", ATTRIBUTETYPE_TEXT),
      ]))
assert(ret)

ensure layer exists

In [5]:
ret = okc.create_layer(okapiclient, layer_id, "just a test layer", okc.hex_string_to_rgb_color("#6666FF"))
assert(ret)

Define pydantic class, same as trait, with the addition of a ciid field

In [6]:
class Test(BaseModel):
    ciid: okc.SerializableUUID # this is the short form for an annotated UUID field that is properly serializable to a string
    id: int
    array: List[str]
    some_key: str

Init data

In [7]:
data_init = [
    Test(ciid=uuid.uuid4(), id=1, array=["a", "b"], some_key="Value 1"), # create our own ciids, uuid4 is recommended
    Test(ciid=uuid.uuid4(), id=3, array=["c", "d"], some_key="another Value"),
]
data_init

[Test(ciid=UUID('fe248e86-0faa-4855-97ed-4384c7a2dcb7'), id=1, array=['a', 'b'], some_key='Value 1'),
 Test(ciid=UUID('8a7cc373-9bba-437d-8031-d5b5a2593de4'), id=3, array=['c', 'd'], some_key='another Value')]

Write data to omnikeeper

In [8]:
ret = okc.bulk_replace_trait_entities_pydantic(okapiclient, trait_name=trait_id, input=data_init, write_layer=layer_id)
assert(ret)

Read data back from omnikeeper, should be the same

In [9]:
m = okc.get_all_traitentities_pydantic(okapiclient, trait_name=trait_id, ta=TypeAdapter(List[Test]), layers=[layer_id])
m

[Test(ciid=UUID('8a7cc373-9bba-437d-8031-d5b5a2593de4'), id=3, array=['c', 'd'], some_key='another Value'),
 Test(ciid=UUID('fe248e86-0faa-4855-97ed-4384c7a2dcb7'), id=1, array=['a', 'b'], some_key='Value 1')]

Change some data locally

In [10]:
m[0].array=['x', 'y']

Write it to omnikeeper and read it again

In [11]:
ret = okc.bulk_replace_trait_entities_pydantic(okapiclient, trait_name=trait_id, input=m, write_layer=layer_id)
assert(ret)
m = okc.get_all_traitentities_pydantic(okapiclient, trait_name=trait_id, ta=TypeAdapter(List[Test]), layers=[layer_id])
m

[Test(ciid=UUID('8a7cc373-9bba-437d-8031-d5b5a2593de4'), id=3, array=['x', 'y'], some_key='another Value'),
 Test(ciid=UUID('fe248e86-0faa-4855-97ed-4384c7a2dcb7'), id=1, array=['a', 'b'], some_key='Value 1')]

## ideas section

In [12]:
# idea for pydantic-model-first generation of ok trait:
# use field annotations to add information necessary to build trait out of model/class
class AnnotatedTest(BaseModel): 
    ciid: okc.SerializableUUID
    id: Annotated[int, okc.AttributeName("test.id")]
    array: Annotated[List[str], okc.AttributeName("test.array")] # array is inferred automatically
    some_key: Annotated[str, okc.AttributeName("test.mapped_some_key"), okc.TypeHint(ATTRIBUTETYPE_TEXT, is_array=False)] # OKTypeHint would not be necessary, but just as an example 
    optional_field: Annotated[int, okc.AttributeName("test.optional_field"), okc.AttributeOptional()] # optional attribute
# TODO: can we also annotate the class itself and add the ok trait name? Or some otherway?

# then you could tell omnikeeper about our trait by just passing the class:
# okc.upsert_trait_from_pydantic_model(okapiclient, trait_id, AnnotatedTest)

In [13]:
# idea for ensuring pydantic model and ok trait are in sync

# okc.check_pydantic_model_ok_trait_synced(trait_id, Test)

# would return bool whether in sync or not -> likely throw error if not

In [14]:
# relations
# relations are hard :(
# relations are the reason ORMs are complex, and I don't want to build an ORM
# try to stay lower level
# give user more control
# make relation loading and saving explicit